# 7.4.1 Training vs Inference

In [45]:
pwd

'C:\\Users\\reeshabh.choudhary\\OneDrive - xpdnp\\Coding\\ML\\Books\\Deep Learning with Python 2nd edition-François Chollet(2021)\\mine\\Chapter 7'

In [1]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt

In [43]:
loss_tracking_metric = keras.metrics.Mean()

In [2]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
        gradients = tape.gradients(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(model.trainable_weights, gradients))

# 7.4.2 Low level usages of metrics

In [3]:
metrics = keras.metrics.SparseCategoricalAccuracy()

In [13]:
targets = [0, 1, 2]
predictions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]

metrics.update_state(targets, predictions)

current_result = metrics.result()

print(f"result: {current_result:.2f}")

result: 1.00


In [12]:
values = [0, 1, 2, 3, 4]
mean_tracker = keras.metrics.Mean()
for value in values:
    mean_tracker.update_state(value)
print(f"Mean of values: {mean_tracker.result():.2f}")

Mean of values: 2.00


# 7.4.3-A complete training and evaluation loop

In [20]:
def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28, ))
    features = keras.layers.Dense(512, activation='relu')(inputs)
    features = keras.layers.Dropout(0.5)(features)
    outputs = keras.layers.Dense(10, activation='softmax')(features)
    model = keras.Model(inputs, outputs)
    return model

In [21]:
from keras.datasets import mnist
(images, labels), (test_images, test_labels) = mnist.load_data()

In [22]:
images = images.reshape((60000, 28 * 28)).astype('float32') / 255

In [23]:
test_images = test_images.reshape((10000, 28 * 28)).astype('float32') / 255

In [24]:
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

In [25]:
import warnings

In [26]:
warnings.filterwarnings(action="ignore", category=DeprecationWarning)

In [27]:
warnings.filterwarnings(action="ignore", category=Warning)

In [28]:
model = get_mnist_model()

In [31]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.RMSprop()
metrics = [keras.metrics.SparseCategoricalAccuracy()]
loss_tracking = keras.metrics.Mean()

In [32]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))
    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs[metrics.name] = metrics.result()
    loss_tracking_metric.update_state(loss)
    logs['loss'] = loss_tracking_metric.result()
    return logs

In [33]:
def reset_metrics():
    for metric in metrics:
        metric.reset_state()
    loss_tracking_metric.reset_state()

In [35]:
training_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels)
)

In [37]:
training_dataset = training_dataset.batch(32)

In [38]:
epochs = 3

In [44]:
for epoch in range(epochs):
    reset_metrics()
    for inputs_batch, targets_batch in training_dataset:
        logs = train_step(inputs_batch, targets_batch)
    print(f"Results at the end of epoch {epoch}")
    for key, value in logs.items():
        print(f"...{key}: {value:.4f}")

ValueError: Input 0 of layer "model" is incompatible with the layer: expected shape=(None, 784), found shape=(32, 2, 784)

In [40]:
def test_step(inputs, targets):
    predictions = model(inputs, training=False)
    loss = loss_fn(targets, predictions)

    logs = {}
    for metric in metrics:
        metric.update_state(targets, predictions)
        logs["val_" + metric.name] = metric.result()
        loss_tracking_metric.update_state(loss)
        logs["val_loss"] = loss_tracking_metric.result()
        return logs

In [41]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()

NameError: name 'loss_tracking_metric' is not defined

In [42]:
for inputs_batch, targets_batch in val_dataset:
    logs = test_step(inputs_batch, targets_batch)
print('Evaluation results:')
for key, value in logs.items():
    print(f"...{key}: {value:.4f}")

NameError: name 'loss_tracking_metric' is not defined